In [1]:
from pathlib import Path


import sys
sys.path.append('..')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from joblib import dump,load
from sklearn.ensemble import RandomForestClassifier
from src.app.preprocess import preprocess
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from pathlib import Path
DATA_DIR = Path(r'../../../AI_Methodology/AI_METHODOLY/data/external/')
PREDICTION_DIR = Path(r'../../../AI_Methodology/AI_METHODOLY/data/Predictions/')
MODEL_DIR=Path(r'../../../AI_Methodology/AI_METHODOLY/models/')
df_master= pd.read_excel(DATA_DIR/"Employee_Perfomance.xls",index_col='EmpNumber')





# Model evaluation and classification Report

In [3]:
def build_model(data: pd.DataFrame):
    target = data['PerformanceRating']
    train = data.drop(['PerformanceRating'], axis=1)
    processedtrain= preprocess(train, MODEL_DIR)
    x_train , x_test , y_train , y_test = train_test_split(processedtrain,target,test_size=0.2)
    rfc = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
    rfc.fit(x_train, y_train)
    dump(rfc,MODEL_DIR/'RFC.joblib')
    predict = rfc.predict(x_test)
    print(f"The accuracy score is : {accuracy_score(y_test, predict) * 100}%")
    print (classification_report(y_test,predict))
    return   

In [4]:
build_model(df_master)

C:\Users\Anand\.conda\envs\aimethodology\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Anand\.conda\envs\aimethodology\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Anand\.conda\envs\aimethodology\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


The accuracy score is : 91.66666666666666%
              precision    recall  f1-score   support

           2       1.00      0.80      0.89        40
           3       0.89      1.00      0.94       168
           4       1.00      0.62      0.77        32

    accuracy                           0.92       240
   macro avg       0.96      0.81      0.87       240
weighted avg       0.93      0.92      0.91       240



# Predictions

In [5]:
def inference(data: pd.DataFrame):
    target = data['PerformanceRating']
    train = data.drop(['PerformanceRating'], axis=1)
    processedtrain= preprocess(train, MODEL_DIR)
    x_train , x_test , y_train , y_test = train_test_split(processedtrain,target,test_size=0.2)
    
    k=load(MODEL_DIR/'RFC.joblib')
    predict = k.predict(x_test)
    return predict

In [6]:
inference_pred=inference(df_master)
inference_pred

C:\Users\Anand\.conda\envs\aimethodology\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Anand\.conda\envs\aimethodology\lib\site-packages\sklearn\utils\validation.py:758: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


array([2, 2, 3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 3,
       3, 3, 3, 3, 3, 4, 3, 4, 2, 3, 3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 4, 3, 3, 3, 3, 2, 3, 3, 4, 3, 3, 3,
       2, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 4, 3, 3, 2,
       2, 3, 3, 4, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 4, 3, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 2, 3, 2, 3, 3, 3, 4, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 2, 2, 2,
       4, 3, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 3, 2, 3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 2, 3],
      dtype=int64)

In [7]:
df = pd.DataFrame(inference_pred, columns = ['perfomance_rating'])
submissiondf=df.rename_axis('Emp_ID').reset_index()

In [8]:
submissiondf

,Emp_ID,perfomance_rating
0,0,2
1,1,2
2,2,3
3,3,3
4,4,3
...,...,...
235,235,3
236,236,3
237,237,3
238,238,2


In [9]:
submissiondf.to_csv(PREDICTION_DIR / 'submission.csv', index=False)